# 🔍 RAG com Gemini e FAISS – Recuperação de Documentos

Este notebook implementa um pipeline RAG (Retrieval-Augmented Generation) usando Gemini como modelo LLM e FAISS como mecanismo de busca vetorial local.

Documentos `.txt` serão carregados de uma pasta, transformados em embeddings, indexados e consultados com base nas perguntas do usuário.


In [1]:
# Instale os pacotes necessários (se ainda não tiver)
# !pip install langchain langchain-google-genai faiss-cpu tiktoken

## 1. Carregar e dividir os documentos

In [7]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
import google.generativeai as genai

load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=api_key)

# Define a pasta com os arquivos .txt
PASTA_DOCS = "meus_arquivos"

# Carrega todos os documentos .txt da pasta
def carregar_documentos(pasta):
    docs = []
    for nome in os.listdir(pasta):
        if nome.endswith(".txt"):
            caminho = os.path.join(pasta, nome)
            loader = TextLoader(caminho, encoding="utf-8")
            docs.extend(loader.load())
    return docs

documentos = carregar_documentos(PASTA_DOCS)
print(f"{len(documentos)} documentos carregados.")

1 documentos carregados.


In [3]:
# Dividir documentos em pedaços menores
splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs_divididos = splitter.split_documents(documentos)
print(f"{len(docs_divididos)} fragmentos gerados.")

1 fragmentos gerados.


## 2. Gerar embeddings e criar índice FAISS

In [9]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS

# Carregar embeddings do Gemini
embeddings = GoogleGenerativeAIEmbeddings(
    google_api_key=api_key,
    model="models/embedding-001"
    )

# Criar vetor FAISS
db = FAISS.from_documents(docs_divididos, embeddings)

## 3. Inicializar o modelo Gemini

In [14]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.5,
    google_api_key=api_key
)

## 4. Construir o Chain de RAG

In [15]:
from langchain.chains import RetrievalQA

# Cria o chain de pergunta-resposta com recuperação
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=db.as_retriever(),
    return_source_documents=True
)

## 5. Fazer perguntas ao sistema

In [18]:
pergunta = "O que o documento diz sobre a lua?"
resposta = rag_chain(pergunta)
print("Resposta:\n", resposta['result'])

Resposta:
 O documento diz que a lua é de queijo.


In [17]:
print("Documentos usados como fonte:")
for doc in resposta['source_documents']:
    print("-", doc.metadata['source'])

Documentos usados como fonte:
- meus_arquivos\fakes_1.txt
